In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

T = pd.read_csv("ghana_targeting.csv")

tmp = pd.concat([ T[['cutoff','i50']],T[['cutoff','e50']] ],sort=False)
tmp['status'] = tmp['i50'].isnull().map({False: "Poor",True:"Non-Poor"})
tmp['Mean'] = tmp[['i50','e50']].sum(1)

# fig2= px.histogram(T,x='e50',nbins=10)#,y='i50')
fig = go.Figure(data=[go.Line(y=T['i50'], orientation='v',x=T['cutoff'], name="Non-Poor",
                             hovertemplate= "Non-Poor Included: <b>%{y:.1%}</b><br><extra></extra>")])
fig.add_trace(go.Line(y=T['e50'], orientation='v',x=T['cutoff'], name="Poor",
        hovertemplate= "Poor Excluded: <b>%{y:.1%}</b><br><extra></extra>"))
fig.update_traces(mode="markers+lines")
fig.update_layout(
    title_text='Predicted Targeting Errors by Cutoff Threshold',
    xaxis_title_text="PPI Score Used as Threshold",
    yaxis_title_text="% of HH's Misclassified",
    hovermode="x unified"
)

/home/elliott/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [3]:
fig.write_html("errors.html")

In [7]:
fig.write_json("errors.json")